# Backtest: Strategy vs Buy & Hold

Compares RSI+MACD and LogReg strategies against Buy & Hold on SPY.

In [ ]:
import sys
sys.path.append('..')

import yfinance as yf
import matplotlib.pyplot as plt
from tradbot.data import YFinanceFix
from tradbot.signals.signal_rsi_macd import generate_strategy as rsi_strategy
from tradbot.signals.signal_logreg import generate_strategy as logreg_strategy
from tradbot.backtest import Backtest

YFinanceFix()

df = yf.download("SPY", period="5y")
df.columns = df.columns.get_level_values(0)
print(f"{len(df)} rows loaded")

## 1. RSI + MACD Strategy

In [ ]:
bt_rsi = Backtest(rsi_strategy(df))
bt_rsi.run()
bt_rsi.print_summary()

In [ ]:
# Plot: Cumulative Returns
result = bt_rsi.get_df()

plt.figure(figsize=(12, 5))
plt.plot(result["asset_cum_returns"], label="Buy & Hold")
plt.plot(result["strategy_cum_returns"], label="RSI + MACD Strategy")
plt.legend()
plt.title("RSI + MACD Strategy vs Buy & Hold (SPY 5Y)")
plt.ylabel("Cumulative Return")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Signal distribution
result["Strategy"].value_counts().sort_index().rename(
    index={-1.0: "Short", 0.0: "Flat", 1.0: "Long"}
)

## 2. Logistic Regression Strategy

In [ ]:
bt_logreg = Backtest(logreg_strategy(df, shift=5, threshold=0.55))
bt_logreg.run()
bt_logreg.print_summary()

In [ ]:
# Plot: Cumulative Returns
result2 = bt_logreg.get_df()

plt.figure(figsize=(12, 5))
plt.plot(result2["asset_cum_returns"], label="Buy & Hold")
plt.plot(result2["strategy_cum_returns"], label="LogReg Strategy")
plt.legend()
plt.title("LogReg Strategy vs Buy & Hold (SPY 5Y)")
plt.ylabel("Cumulative Return")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Signal distribution (LogReg)
result2["Strategy"].value_counts().sort_index().rename(
    index={-1.0: "Short", 0.0: "Flat", 1.0: "Long"}
)

## 3. Comparison: Both Strategies

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(result["asset_cum_returns"], label="Buy & Hold", linewidth=2)
plt.plot(result["strategy_cum_returns"], label="RSI + MACD", linestyle="--")
plt.plot(result2["strategy_cum_returns"], label="LogReg", linestyle="--")
plt.legend()
plt.title("Strategy Comparison (SPY 5Y)")
plt.ylabel("Cumulative Return")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()